In [4]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from nltk.stem import WordNetLemmatizer as Lemmatizer
import numpy as np
import re
from src.logit_gb import LogitGB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [20]:
with open('data/text.p', 'rb') as file:
    text = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)
with open('data/predictor.p', 'rb') as file:
    pred = pickle.load(file)

In [18]:
pred_train = pred[pred['status_date'].apply(lambda x: x.year < 2015)]

In [25]:
with open('models/model_up.p', 'rb') as file:
    model_up = pickle.load(file)
with open('models/vec.p', 'rb') as file:
    vec = pickle.load(file)

In [13]:
with open('train/fires_before2015.p', 'rb') as file:
    fires = pickle.load(file)

In [33]:
with open('models/vec.p', 'wb') as file:
    pickle.dump(vec, file)

In [34]:
with open('train/fires.p', 'rb') as file:
    fires = pickle.load(file)

In [32]:
pred.shape

(423099, 51)

In [19]:
pred_train['fires_before'] = fires[:,0]
pred_train['fires_after'] = fires[:,1]
pred_train['fire_rate_before'] = 365*pred['fires_before']/(pred['status_date'] - pd.to_datetime('2003-01-01')).apply(lambda x: x.days)
pred_train['fire_rate_after'] = 365*pred['fires_after']/(pd.to_datetime('2014-12-31') - pred['status_date']).apply(lambda x: x.days)

In [36]:
pred['fires_before'] = fires[:,0]
pred['fires_after'] = fires[:,1]
pred['fire_rate_before'] = 365*pred['fires_before']/(pred['status_date'] - pd.to_datetime('2003-01-01')).apply(lambda x: x.days)
pred['fire_rate_after'] = 365*pred['fires_after']/(pd.to_datetime('2019-02-28') - pred['status_date']).apply(lambda x: x.days)

In [17]:
with open('models/pred_train.p', 'wb') as file:
    pickle.dump(pred, file)

In [22]:
train_mask = pred['status_date'].apply(lambda x: x.year < 2015).values

In [26]:
X_text_train = vec.transform(text[train_mask])

In [79]:
text_train = pred_train[['existing_use', 'proposed_use', 'description']].fillna('').astype(str).apply(lambda x: ' '.join(x), axis = 1)

In [21]:
def hasnumber(s):
    return bool(re.search(r'\d', s))

In [88]:
text_train = np.array([' '.join([Lemmatizer.lemmatize('n', word.lower()) for word in doc.split() if not hasnumber(word)]) for doc in text_train])

In [167]:
pred_train = pred[pred['status_date'].apply(lambda x: x.year < 2015)]

In [4]:
with open('locations.p', 'rb') as file:
    locations = pickle.load(file)

In [57]:
with open('models/model_down.p', 'rb') as file:
    model = pickle.load(file)

In [50]:
pred_train['has_existing_stories'] = (~pred_train['existing_stories'].isnull()).astype(int)
pred_train['existing_stories'] = pred_train['existing_stories'].fillna(0)

C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
from src.logit_gb import LogitGB

In [82]:
sample = np.random.choice(X_text_train.shape[0], 15000, replace = False)

In [83]:
model_up = LogitGB(columns = ['existing_stories', 'has_existing_stories'])
model_up.fit(X_text_train[sample], pred_train.iloc[sample], pred_train.iloc[sample], any = True)

C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [54]:
test_mask = pred['status_date'].apply(lambda x: x.year < 2016).values
test_mask = test_mask & ~train_mask

In [40]:
train_mask

array([ True,  True,  True, ..., False, False, False])

In [43]:
X_text = vec.transform(text)

In [59]:
with open('data/y_increase.p', 'wb') as file:
    pickle.dump(y, file)

In [118]:
with open('train/test_mask.p', 'wb') as file:
    pickle.dump(test_mask, file)

In [58]:
with open('models/model_up.p', 'wb') as file:
    pickle.dump(model_up, file)

In [37]:
y = (pred['fire_rate_after'] > 0).astype(int)

In [124]:
y_test = (pred[test_mask]['fire_rate_after'] > pred[test_mask]['fire_rate_before']).astype(int)

In [84]:
roc_auc_score(y[test_mask], model_up.predict_proba(pred[test_mask], X_text[test_mask]))

0.806558480797639

In [39]:
def f1(y_test, y_pred, threshold):
    return f1_score(y_test, (y_pred > threshold).astype(int))

In [52]:
1 - y[test_mask].mean()

0.8795593470813461

In [148]:
maj_preds[0] = 1

In [90]:
f1_score(y[test_mask], model_up.predict(pred[test_mask], X_text[test_mask]))

0.4337857666911225

In [92]:
pred_train['location'] = pred_train['address'].apply(lambda x: locations[x])

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [62]:
vocab_idx = {value:key for (key, value) in vec.vocabulary_.items()}

In [91]:

words2

[('tower', 2.119859556056828),
 ('damage', 2.030497090895514),
 ('office', 2.014715783457073),
 ('demo', 1.9073607745076757),
 ('strobe', 1.8123676442488195),
 ('apt', 1.7455522872963423),
 ('suite', 1.7157070947941861),
 ('live', 1.4953086644025981),
 ('penthouse', 1.4733441847193836),
 ('hotel', 1.4156537680600147),
 ('artist', 1.412454585761788),
 ('expansion', 1.3611830814688466),
 ('panels', 1.2803054490193073),
 ('menan', 1.2757054197602788),
 ('grid', 1.272272361893477),
 ('removal', 1.257209491749912),
 ('corridor', 1.2535667347181292),
 ('include', 1.2499717446827938),
 ('server', 1.2088303427594862),
 ('sp', 1.2081364180351493),
 ('hospital', 1.1860360882764724),
 ('millwork', 1.1834097155052032),
 ('space', 1.1785618891208407),
 ('sale', 1.1507008646256984),
 ('safety', 1.1480235213176386),
 ('units', 1.1393250701312838),
 ('illuminated', 1.1325127662415029),
 ('sprinkler', 1.131408145175543),
 ('exhaust', 1.1191821599948506),
 ('fire', 1.1182963521410052)]

In [86]:
[word for word in np.array(words1)[:,0] if word in np.array(words2)[:,0]]

['damage',
 'hotel',
 'fire',
 'office',
 'suite',
 'apt',
 'hospital',
 'space',
 'artist',
 'live',
 'tower']

In [96]:
pred[pred['description'].astype(str).apply(lambda x: 'OFFICE' in x)]['description'].values

array(['CHANGE BDRM TO OFFICE USE ON 1ST FLOOR IN UNIT #149                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    nan',
       'CHANGE BDRM TO OFFICE USE ON 1ST FLOOR IN UNIT #149                                                                                                                                        

In [136]:
pred_trim['has_existing_stories'] = (pred_trim['existing_stories']>0).astype(int)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [118]:
X_text_test = vec.transform(text_test)

In [120]:
pred_trim = pred[['existing_stories', 'address', 'status_date', 'location']]

In [121]:
pred_trim_test = pred_trim[~train_mask.values]

In [126]:
pred_trim_test['has_existing_stories'] = (pred_trim_test['existing_stories']>0).astype(int)
pred_trim_test['existing_stories'].fillna(0, inplace = True)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\pengj\Anaconda2\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [55]:
from sklearn.metrics import roc_auc_score

In [127]:
roc_auc_score(y_test, model_up.predict_proba(pred_trim_test, X_text_test))

0.8429165983004447

In [113]:
with open('models/model_up.p', 'wb') as file:
    pickle.dump(model_up, file)
with open('models/vec.p', 'wb') as file:
    pickle.dump(vec, file)

model_up.predict(pred_trim, text.reshape(-1,1))